In [1]:
import tensorflow as tf
import numpy as np
import os, sys
import time
from tqdm import tqdm
import time
import pickle
import gensim

In [2]:
with open('../data/wikinews/wiki_train.pkl', 'rb') as f :
    train_data, seq_length = pickle.load(f)

In [3]:
with open('../data/wikinews/word_vocab.pkl', 'rb') as f :
    word_vocab = pickle.load(f)

In [4]:
vocab_size = len(word_vocab) - 1

In [5]:
batch_size = 256

In [6]:
num_units = 256

In [7]:
mode = 'self'

In [8]:
with tf.variable_scope('model') :
    train_x, train_y = train_data[:256], train_data[:256]
    output_lengths = input_lengths = tf.to_int32(seq_length[:256])

In [9]:
input_embed = tf.contrib.layers.embed_sequence(
        train_x, vocab_size = vocab_size, embed_dim = num_units, scope='embed')
output_embed = tf.contrib.layers.embed_sequence(
        train_y, vocab_size = vocab_size, embed_dim = num_units, scope='embed', reuse=True)
with tf.variable_scope('embed', reuse=True):
    embeddings = tf.get_variable('embeddings')

In [10]:
cell = tf.contrib.rnn.LSTMCell(num_units)
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(cell, input_embed, dtype = tf.float32)
train_helper = tf.contrib.seq2seq.TrainingHelper(output_embed, output_lengths)

In [11]:
pred_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
    embeddings, start_tokens = tf.to_int32([3] * batch_size), end_token = 4)

In [12]:
def decode(helper, scope_name, reuse = tf.AUTO_REUSE) :
    with tf.variable_scope(scope_name, reuse = reuse):
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
            num_units = num_units, memory = encoder_outputs,
            memory_sequence_length = input_lengths)
        cell = tf.contrib.rnn.LSTMCell(num_units)
        attn_cell = tf.contrib.seq2seq.AttentionWrapper(
            cell, attention_mechanism, attention_layer_size = num_units / 2)
        out_cell = tf.contrib.rnn.OutputProjectionWrapper(
            attn_cell, vocab_size, reuse = reuse
        )
        decoder = tf.contrib.seq2seq.BasicDecoder(
            cell = out_cell, helper = helper,
            initial_state = out_cell.zero_state(
                dtype = tf.float32, batch_size = batch_size))
#             initial_state = encoder_state)
        outputs = tf.contrib.seq2seq.dynamic_decode(
            decoder = decoder, output_time_major = False,
            impute_finished = True, maximum_iterations = 30
        )
    return outputs[0]

In [22]:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.001
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100, 0.95, staircase = True)

In [26]:
train_outputs = decode(train_helper, 'decode')
pred_outputs = decode(pred_helper, 'decode', reuse = True)

tf.identity(train_outputs.sample_id[0], name = 'train_pred')
weights = tf.to_float(tf.not_equal(train_y, 1))
loss = tf.contrib.seq2seq.sequence_loss(
    train_outputs.rnn_output, tf.to_int32(train_y), weights = weights)
train_op = tf.contrib.layers.optimize_loss(
    loss, tf.train.get_global_step(),
    optimizer = 'Adam',
    learning_rate = learning_rate,
    summaries = ['loss', 'learning_rate'])

tf.identity(pred_outputs.sample_id[0], name='predictions')

<tf.Tensor 'predictions_1:0' shape=(?,) dtype=int32>

In [27]:
tf.estimator.EstimatorSpec(
    mode = mode,
    predictions = pred_outputs.sample_id,
    loss = loss,
    train_op = train_op)

EstimatorSpec(mode='self', predictions=<tf.Tensor 'decode_11/decoder/transpose_1:0' shape=(256, ?) dtype=int32>, loss=<tf.Tensor 'sequence_loss_5/truediv:0' shape=() dtype=float32>, train_op=<tf.Tensor 'OptimizeLoss_1/control_dependency:0' shape=() dtype=float32>, eval_metric_ops={}, export_outputs=None, training_chief_hooks=(), training_hooks=(), scaffold=<tensorflow.python.training.monitored_session.Scaffold object at 0x7f83e39b7438>, evaluation_hooks=(), prediction_hooks=())